In [1]:
import pandas as pd
import sqlite3

con = sqlite3.connect('Data.sqlite')
dfs = pd.read_excel('Data.xlsx',sheet_name=None)
for table, df in dfs.items():
    df.to_sql(table, con,if_exists='replace',index=False)
con.commit()
con.close()

In [2]:
%load_ext sql
%sql sqlite:///Data.sqlite
#%sql postgresql:///postgres:password@localhost:5432/postgres
# Turn off echo of connection information
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

## UNION vs UNION ALL

|UNION ALL|UNION|
|---|---| 
|selects all the values|only distinct values are selected (eliminate duplicate rows)|
|faster|slower|

## HAVING vs WHERE Clause

|HAVING|WHERE|
|---|---|
|can be used with aggregates|works on row’s data, not on aggregated data|

In [3]:
%%sql
SELECT name
FROM sqlite_schema
WHERE type ='table' AND name NOT LIKE 'sqlite_%';

name
yelp_business
db_employee
db_dept
fb_friend_requests
fb_eu_energy
fb_asia_energy
fb_na_energy
amazon_transactions
facebook_friends
customers


## 1. Salaries Differences
Write a query that calculates the difference between the highest salaries found in the marketing and engineering departments. Output just the absolute difference in salaries.

In [4]:
%%sql
WITH table1 AS (select *
                from db_employee e1
                join db_dept e2
                on e1.department_id = e2.id)
select abs(max(case when department = 'marketing' then salary end) - max(case when department = 'engineering' then salary end)) salary_difference
from table1

salary_difference
2400


## 2. Acceptance Rate By Date
What is the overall friend acceptance rate by date? Your output should have the rate of acceptances by the date the request was sent. Order by the earliest date to latest.

Assume that each friend request starts by a user sending (i.e., user_id_sender) a friend request to another user (i.e., user_id_receiver) that's logged in the table with action = 'sent'. If the request is accepted, the table logs action = 'accepted'. If the request is not accepted, no record of action = 'accepted' is logged.

In [5]:
%%sql
with table1 as (SELECT date,
                    user_id_sender,
                    user_id_receiver
                FROM fb_friend_requests
                WHERE action='sent'),
    table2 as (SELECT date,
                    user_id_sender,
                    user_id_receiver
               FROM fb_friend_requests
               WHERE action='accepted')
               
select t1.date,
        ROUND(cast(count(t2.user_id_receiver) AS float)/cast(count(t1.user_id_sender) AS float),2) AS percentage_acceptance
from table1 t1
left join table2 t2
on t1.user_id_sender = t2.user_id_sender
and t1.user_id_receiver = t2.user_id_receiver
group by t1.date  

date,percentage_acceptance
2020-01-04 00:00:00,0.75
2020-01-06 00:00:00,0.67


## 3. Highest Energy Consumption
Find the date with the highest total energy consumption from the Facebook data centers. Output the date along with the total energy consumption across all data centers.

In [6]:
%%sql
WITH total_energy AS (SELECT *
                    FROM fb_eu_energy eu
                    UNION ALL SELECT *
                    FROM fb_asia_energy asia
                    UNION ALL SELECT *
                    FROM fb_na_energy na),
     energy_by_date AS (SELECT date, sum(consumption) AS total_energy
                        FROM total_energy
                        GROUP BY date
                        ORDER BY date ASC),
     max_energy AS (SELECT max(total_energy) AS max_energy
                    FROM energy_by_date)

SELECT  strftime('%Y-%m-%d', t1.date) date1,
        date(t1.date) date2,
        t1.date,
       t1.total_energy
FROM energy_by_date t1
JOIN max_energy t2 
ON t1.total_energy = t2.max_energy

date1,date2,date,total_energy
2020-01-06,2020-01-06,2020-01-06 00:00:00,1250
2020-01-07,2020-01-07,2020-01-07 00:00:00,1250


In [7]:
%%sql
WITH total_energy AS (SELECT *
                    FROM fb_eu_energy eu
                    UNION ALL SELECT *
                    FROM fb_asia_energy asia
                    UNION ALL SELECT *
                    FROM fb_na_energy na),
        energy_rank AS (SELECT date,
                                sum(consumption) AS total_energy,
                                rank() OVER (ORDER BY sum(consumption) DESC) AS rnk
                        FROM total_energy
                        GROUP BY date
                        ORDER BY date ASC)
SELECT  date(date) date,
        total_energy
FROM energy_rank
WHERE rnk = 1;


date,total_energy
2020-01-06,1250
2020-01-07,1250


## 4. Finding User Purchases
Write a query that'll identify returning active users. A returning active user is a user that has made a second purchase within 7 days of any other of their purchases. Output a list of user_ids of these returning active users.

In [8]:
%%sql
SELECT  t1.user_id,
        t1.created_at created_at_1st,
        t2.created_at created_at_2nd,
        JulianDay(t2.created_at) - JulianDay(t1.created_at) Diff_Days,
        (JulianDay(t2.created_at) - JulianDay(t1.created_at)) * 24 Diff_Hours,
        (JulianDay(t2.created_at) - JulianDay(t1.created_at)) * 24 * 60 Diff_Minutes,
        (JulianDay(t2.created_at) - JulianDay(t1.created_at)) * 24 * 60 * 60 Diff_Seconds
FROM amazon_transactions t1
JOIN amazon_transactions t2 
ON t1.user_id = t2.user_id and t1.created_at != t2.created_at and Diff_Days BETWEEN 0 AND 7
ORDER BY 1

user_id,created_at_1st,created_at_2nd,Diff_Days,Diff_Hours,Diff_Minutes,Diff_Seconds
100,2020-03-13 00:00:00,2020-03-18 00:00:00,5.0,120.0,7200.0,432000.0
100,2020-03-07 00:00:00,2020-03-13 00:00:00,6.0,144.0,8640.0,518400.0
103,2020-03-29 00:00:00,2020-03-31 00:00:00,2.0,48.0,2880.0,172800.0
105,2020-03-05 00:00:00,2020-03-09 00:00:00,4.0,96.0,5760.0,345600.0
109,2020-03-02 00:00:00,2020-03-03 00:00:00,1.0,24.0,1440.0,86400.0
111,2020-03-19 00:00:00,2020-03-23 00:00:00,4.0,96.0,5760.0,345600.0
111,2020-03-18 00:00:00,2020-03-19 00:00:00,1.0,24.0,1440.0,86400.0
111,2020-03-18 00:00:00,2020-03-23 00:00:00,5.0,120.0,7200.0,432000.0
112,2020-03-23 00:00:00,2020-03-24 00:00:00,1.0,24.0,1440.0,86400.0
114,2020-03-25 00:00:00,2020-03-31 00:00:00,6.0,144.0,8640.0,518400.0


In [9]:
%%sql
SELECT DISTINCT(t1.user_id)
FROM amazon_transactions t1
JOIN amazon_transactions t2 
ON t1.user_id = t2.user_id and t1.created_at != t2.created_at and JulianDay(t2.created_at) - JulianDay(t1.created_at) BETWEEN 0 AND 7
ORDER BY 1

user_id
100
103
105
109
111
112
114
117
120
122


## 5. Popularity Percentage
Find the popularity percentage for each user on Facebook. The popularity percentage is defined as the total number of friends the user has divided by the total number of users on the platform, then converted into a percentage by multiplying by 100.
Output each user along with their popularity percentage. Order records in ascending order by user id.
The 'user1' and 'user2' column are pairs of friends.

In [10]:
%%sql
SELECT  user1,
        user2
FROM facebook_friends
UNION
SELECT  user2 AS user1,
        user1 AS user2
FROM facebook_friends

user1,user2
1,2
1,3
1,4
1,5
1,6
2,1
2,6
2,7
3,1
3,8


In [11]:
%%sql
WITH table1 AS (SELECT  user1,
                        user2
                FROM facebook_friends
                UNION
                SELECT  user2 AS user1,
                        user1 AS user2
                FROM facebook_friends)
SELECT count(DISTINCT user1) AS users_count
FROM table1

users_count
9


In [12]:
%%sql
WITH table1 AS (SELECT  user1,
                        user2
                FROM facebook_friends
                UNION
                SELECT  user2 AS user1,
                        user1 AS user2
                FROM facebook_friends),
    table2 AS (SELECT count(DISTINCT user1) AS users_count
                FROM table1)
SELECT t1.user1,
       ROUND(CAST(COUNT(*) AS float)*100/CAST(t2.users_count AS float),2) AS popularity_percentage
FROM    table1 t1,
        table2 t2
GROUP BY 1
ORDER BY 1

user1,popularity_percentage
1,55.56
2,33.33
3,33.33
4,11.11
5,11.11
6,22.22
7,11.11
8,11.11
9,11.11


In [13]:
%%sql
WITH table1 AS (SELECT  user1,
                        user2
                FROM facebook_friends
                UNION
                SELECT  user2 AS user1,
                        user1 AS user2
                FROM facebook_friends)
SELECT user1,
       ROUND(CAST(COUNT(*) AS float)*100/CAST(SUM(COUNT(DISTINCT user1)) OVER() AS float),2) AS popularity_percentage,
       SUM(COUNT(DISTINCT user1)) OVER() AS users_count
FROM table1
GROUP BY 1
ORDER BY 1

user1,popularity_percentage,users_count
1,55.56,9
2,33.33,9
3,33.33,9
4,11.11,9
5,11.11,9
6,22.22,9
7,11.11,9
8,11.11,9
9,11.11,9


## 6. Highest Cost Orders
Find the customer with the highest daily total order cost between 2019-02-01 to 2019-05-01. If customer had more than one order on a certain day, sum the order costs on daily basis. Output their first name, total cost of their items, and the date.
 
 For simplicity, you can assume that every first name in the dataset is unique.

In [14]:
%%sql
WITH table1 AS (SELECT t1.first_name,
                        DATE(t2.order_date) order_date,
                        SUM(t2.total_order_cost) total_order_cost,
                        RANK() OVER(ORDER BY SUM(t2.total_order_cost) DESC) AS rnk
                FROM orders t2
                LEFT JOIN customers t1
                ON t1.id = t2.cust_id
                WHERE DATE(t2.order_date) BETWEEN '2019-02-01' AND '2019-05-01'
                GROUP BY 1,2)
SELECT first_name,
        total_order_cost,
        order_date
FROM table1
WHERE rnk = 1

first_name,total_order_cost,order_date
Jill,275,2019-04-19


## 7. Users By Avg Session time
Calculate each user's average session time. A session is defined as the time difference between a page_load and page_exit. For simplicity, assume an user has only 1 session per day and if there are multiple of the same events in that day, consider only the latest page_load and earliest page_exit. Output the user_id and their average session time.

In [15]:
%%sql
SELECT *
FROM facebook_web_log

user_id,timestamp,action
0,2019-04-25 13:30:15,page_load
0,2019-04-25 13:30:18,page_load
0,2019-04-25 13:30:40,scroll_down
0,2019-04-25 13:30:45,scroll_up
0,2019-04-25 13:31:10,scroll_down
0,2019-04-25 13:31:25,scroll_down
0,2019-04-25 13:31:40,page_exit
1,2019-04-25 13:40:00,page_load
1,2019-04-25 13:40:10,scroll_down
1,2019-04-25 13:40:15,scroll_down


In [16]:
%%sql
WITH table1 AS (SELECT *
                FROM facebook_web_log
                where action = 'page_load'),
    table2 AS (SELECT *
                FROM facebook_web_log
                where action = 'page_exit')
SELECT t1.user_id,
        t1.timestamp,
        t2.timestamp,
        (JulianDay(t2.timestamp) - JulianDay(t1.timestamp)) * 24 * 60 * 60 AS session_duration
FROM table1 t1
JOIN table2 t2
ON t1.user_id = t2.user_id AND t2.timestamp > t1.timestamp
Group by 1,2

user_id,timestamp,timestamp_1,session_duration
0,2019-04-25 13:30:15,2019-04-25 13:31:40,84.99998152256012
0,2019-04-25 13:30:18,2019-04-25 13:31:40,81.99999779462814
0,2019-04-28 14:30:10,2019-04-28 15:31:40,3689.999981224537
0,2019-04-28 14:30:15,2019-04-28 15:31:40,3685.0000083446503
1,2019-04-25 13:40:00,2019-04-25 13:40:35,35.00001132488251
1,2019-04-26 11:15:00,2019-04-26 11:15:35,35.00001132488251


In [17]:
%%sql
SELECT user_id, 
        DATE(timestamp) AS date,
        MAX(timestamp) AS last_page_load
FROM facebook_web_log
WHERE action = 'page_load'
GROUP BY 1, 2

user_id,date,last_page_load
0,2019-04-25,2019-04-25 13:30:18
0,2019-04-28,2019-04-28 14:30:15
1,2019-04-25,2019-04-25 13:40:00
1,2019-04-26,2019-04-26 11:15:00
2,2019-04-25,2019-04-25 13:41:21


In [18]:
%%sql
WITH daily_start AS (SELECT user_id, 
                            DATE(timestamp) AS date,
                            MAX(timestamp) AS last_page_load
                    FROM facebook_web_log
                    WHERE action = 'page_load'
                    GROUP BY 1, 2), 
    daily_end AS (SELECT user_id, 
                        DATE(timestamp) AS date,
                        MIN(timestamp) AS first_page_exit
                    FROM facebook_web_log
                    WHERE action = 'page_exit'
                    GROUP BY 1,2)

SELECT t1.user_id, 
        AVG(JulianDay(t2.first_page_exit) - JulianDay(t1.last_page_load))*24*60*60 AS avg_session_time
FROM daily_start t1
JOIN daily_end t2
ON t1.user_id = t2.user_id AND t1.date = t2.date
GROUP BY 1

user_id,avg_session_time
0,1883.5000030696392
1,35.00001132488251


In [19]:
%%sql
with table1 AS (SELECT t1.user_id,
                        t1.timestamp,
                        (min(JulianDay(t2.timestamp)) - max(JulianDay(t1.timestamp)))*24*60*60 as session_duration
                FROM facebook_web_log t1
                JOIN facebook_web_log t2
                ON t1.user_id = t2.user_id
                WHERE t1.action = 'page_load' AND t2.action = 'page_exit' AND t2.timestamp > t1.timestamp
                GROUP BY 1, 2)
SELECT user_id,
        AVG(session_duration) AS _avg
FROM table1
GROUP BY 1

user_id,_avg
0,1885.4999922215939
1,35.00001132488251


## 8. Monthly Percentage Difference
Given a table of purchases by date, calculate the month-over-month percentage change in revenue. The output should include the year-month date (YYYY-MM) and percentage change, rounded to the 2nd decimal point, and sorted from the beginning of the year to the end of the year.
The percentage change column will be populated from the 2nd month forward and can be calculated as ((this month's revenue - last month's revenue) / last month's revenue)*100.

In [20]:
%%sql
select * 
from sf_transactions
limit 10;

id,created_at,value,purchase_id
1,2019-01-01 00:00:00,172692,43
2,2019-01-05 00:00:00,177194,36
3,2019-01-09 00:00:00,109513,30
4,2019-01-13 00:00:00,164911,30
5,2019-01-17 00:00:00,198872,39
6,2019-01-21 00:00:00,184853,31
7,2019-01-25 00:00:00,186817,26
8,2019-01-29 00:00:00,137784,22
9,2019-02-02 00:00:00,140032,25
10,2019-02-06 00:00:00,116948,43


In [21]:
%%sql
select strftime('%Y-%m',created_at) AS year_month,
        round(cast(sum(value) - lag(sum(value),1) over(order by strftime('%Y-%m',created_at) asc) as float)/cast(lag(sum(value),1) over(order by strftime('%Y-%m',created_at) asc) as float)*100,2) as revenue_diff_pct
from sf_transactions
group by 1
order by 1

year_month,revenue_diff_pct
2019-01,None
2019-02,-28.56
2019-03,23.35
2019-04,-13.84
2019-05,13.49
2019-06,-2.78
2019-07,-6.0
2019-08,28.36
2019-09,-4.97
2019-10,-12.68


In [22]:
%%sql
with table1 AS (select strftime('%Y-%m',created_at) AS year_month,
                        sum(value) AS current_revenue,
                        lag(sum(value),1) over (order by strftime('%Y-%m',created_at) asc) as prev_revenue
                from sf_transactions
                group by 1
                order by 1)
select year_month,
       round(cast(current_revenue - prev_revenue as float)/cast(prev_revenue as float)*100,2) AS revenue_diff_pct
from table1

year_month,revenue_diff_pct
2019-01,None
2019-02,-28.56
2019-03,23.35
2019-04,-13.84
2019-05,13.49
2019-06,-2.78
2019-07,-6.0
2019-08,28.36
2019-09,-4.97
2019-10,-12.68


## 9. Premium vs Freemium
Find the total number of downloads for paying and non-paying users by date. Include only records where non-paying customers have more downloads than paying customers. The output should be sorted by earliest date first and contain 3 columns date, non-paying downloads, paying downloads.

In [23]:
%%sql
select t1.user_id,
        t1.acc_id,
        DATE(t2.date) AS date,
        t2.downloads,
        t3.paying_customer
from ms_user_dimension t1
left join ms_download_facts t2
on t1.user_id = t2.user_id
left join ms_acc_dimension t3
on t3.acc_id = t1.acc_id

user_id,acc_id,date,downloads,paying_customer
1,716,2020-08-24,6,no
2,749,2020-08-22,6,yes
3,713,2020-08-18,2,no
4,744,2020-08-24,4,yes
5,726,2020-08-19,7,yes
6,706,2020-08-21,3,no
7,750,2020-08-24,1,yes
8,732,2020-08-24,8,yes
9,706,2020-08-17,5,no
10,729,2020-08-16,4,yes


In [24]:
%%sql
select DATE(t2.date) AS date,
        sum(case when t3.paying_customer = 'no' then t2.downloads ELSE 0 end) as non_paying,
        sum(case when t3.paying_customer = 'yes' then t2.downloads ELSE 0 end) as paying
from ms_user_dimension t1
left join ms_download_facts t2
on t1.user_id = t2.user_id
left join ms_acc_dimension t3
on t3.acc_id = t1.acc_id
group by 1
HAVING non_paying > paying
order by 1

date,non_paying,paying
2020-08-16,15,14
2020-08-17,45,9
2020-08-18,10,7
2020-08-21,32,17


In [25]:
%%sql
with table1 AS (select DATE(t2.date) AS date,
                        sum(case when t3.paying_customer = 'no' then t2.downloads ELSE 0 end) as non_paying,
                        sum(case when t3.paying_customer = 'yes' then t2.downloads ELSE 0 end) as paying
                from ms_user_dimension t1
                left join ms_download_facts t2
                on t1.user_id = t2.user_id
                left join ms_acc_dimension t3
                on t3.acc_id = t1.acc_id
                group by 1
                order by 1)
select *
from table1
where non_paying > paying

date,non_paying,paying
2020-08-16,15,14
2020-08-17,45,9
2020-08-18,10,7
2020-08-21,32,17


## 10. Top 5 States With 5 Star Businesses
Find the top 5 states with the most 5 star businesses. Output the state name along with the number of 5-star businesses and order records by the number of 5-star businesses in descending order. In case there are ties in the number of businesses, return all the unique states. If two states have the same result, sort them in alphabetical order.

In [26]:
%%sql
SELECT * 
FROM yelp_business
LIMIT 3;

business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
G5ERFWvPfHy7IDAUYlWL2A,All Colors Mobile Bumper Repair,None,7137 N 28th Ave,Phoenix,AZ,85051,33.448,-112.074,1.0,4,1,Auto Detailing;Automotive
0jDvRJS-z9zdMgOUXgr6rA,Sunfare,None,811 W Deer Valley Rd,Phoenix,AZ,85027,33.683,-112.085,5.0,27,1,Personal Chefs;Food;Gluten-Free;Food Delivery Services;Event Planning & Services;Restaurants
6HmDqeNNZtHMK0t2glF_gg,Dry Clean Vegas,Southeast,"2550 Windmill Ln, Ste 100",Las Vegas,NV,89123,36.042,-115.118,1.0,4,1,Dry Cleaning & Laundry;Laundry Services;Local Services;Dry Cleaning


In [27]:
%%sql
WITH table1 AS (SELECT state,
                        SUM(CASE WHEN stars = 5 THEN 1 ELSE 0 END) AS n_businesses,
                        RANK() OVER (ORDER BY SUM(CASE WHEN stars = 5 THEN 1 ELSE 0 END) DESC) AS rnk
                FROM yelp_business
                GROUP BY 1)

SELECT state,
        n_businesses,
        rnk
FROM table1
WHERE rnk <= 5
ORDER BY 2 DESC,
        1 ASC;

state,n_businesses,rnk
AZ,10,1
ON,5,2
NV,4,3
IL,3,4
OH,3,4
WI,3,4


In [28]:
%%sql
WITH table1 AS (SELECT state,
                    count(business_id) AS n_businesses
            FROM yelp_business
            WHERE stars = 5
            GROUP BY state),
      table2 AS (SELECT *,
                  rank() OVER (ORDER BY n_businesses DESC) AS rnk
                  FROM table1)
SELECT state,
       n_businesses
FROM table2
WHERE rnk <= 5
ORDER BY 2 DESC,
         1 ASC

state,n_businesses
AZ,10
ON,5
NV,4
IL,3
OH,3
WI,3


## 11. Finding Updated Records
We have a table with employees and their salaries, however, some of the records are old and contain outdated salary information. Find the current salary of each employee assuming that salaries increase each year. Output their id, first name, last name, department ID, and current salary. Order your list by employee ID in ascending order.

In [29]:
%%sql
SELECT id,
        first_name,	
        last_name,
        department_id,
        MAX(salary) current_salary
FROM ms_employee_salary
GROUP BY 1,2,3,4
ORDER BY 1 ASC

id,first_name,last_name,department_id,current_salary
1,Todd,Wilson,1006,110000
2,Justin,Simon,1005,130000
3,Kelly,Rosario,1002,42689
4,Patricia,Powell,1004,170000
5,Sherry,Golden,1002,44101
6,Natasha,Swanson,1005,90000
7,Diane,Gordon,1002,74591
8,Mercedes,Rodriguez,1005,61048
9,Christy,Mitchell,1001,150000
10,Sean,Crawford,1006,190000


## 12. Marketing Campaign Success [Advanced]
You have a table of in-app purchases by user. Users that make their first in-app purchase are placed in a marketing campaign where they see call-to-actions for more in-app purchases. Find the number of users that made additional in-app purchases due to the success of the marketing campaign.

The marketing campaign doesn't start until one day after the initial in-app purchase so users that make multiple purchases on the same day do not count, nor do we count users that make only the same purchases over time.

In [30]:
%%sql
SELECT user_id, 
        min(Date(created_at)) over (Partition By user_id) as first_order,
        min(Date(created_at)) over(partition by user_id,product_id) as prod_first_order
FROM marketing_campaign

user_id,first_order,prod_first_order
10,2019-01-01,2019-01-01
10,2019-01-01,2019-03-31
10,2019-01-01,2019-01-02
11,2019-01-02,2019-01-02
11,2019-01-02,2019-03-31
12,2019-01-02,2019-03-31
12,2019-01-02,2019-01-02
13,2019-01-05,2019-01-05
13,2019-01-05,2019-03-31
14,2019-01-06,2019-01-06


In [31]:
%%sql 
WITH table1 AS (SELECT user_id, 
                        min(Date(created_at)) over (Partition By user_id) as first_order,
                        min(Date(created_at)) over(partition by user_id,product_id) as prod_first_order
                FROM marketing_campaign)

select count(distinct user_id) count
from table1
where first_order != prod_first_order

count
23


## 13. Number Of Bathrooms And Bedrooms
Find the average number of bathrooms and bedrooms for each city’s property types. Output the result along with the city name and the property type.

In [32]:
%%sql
select city,
        property_type,
        round(avg(bathrooms),2) n_bathrooms_avg,
        round(avg(bedrooms),2) n_bedrooms_avg
from airbnb_search_details
group by 1,2
order by 1,2

city,property_type,n_bathrooms_avg,n_bedrooms_avg
Boston,Apartment,1.0,1.0
Boston,Condominium,2.0,2.0
Boston,House,1.0,1.0
Chicago,Apartment,2.0,2.0
Chicago,Condominium,2.0,2.0
Chicago,House,2.0,3.0
DC,House,1.0,1.0
LA,Apartment,1.14,1.14
LA,Bungalow,1.0,0.0
LA,Cabin,3.0,1.0


## 14. Customer Details
Find the details of each customer regardless of whether the customer made an order. Output the customer's first name, last name, and the city along with the order details.
You may have duplicate rows in your results due to a customer ordering several of the same items. Sort records based on the customer's first name and the order details in ascending order.
Tables: customers, orders

In [33]:
%%sql
select first_name,
	    last_name,
        city,	
        order_details
from customers t1
left join orders t2
on t1.id = t2.cust_id
order by 1,4 asc

first_name,last_name,city,order_details
Emma,Isaac,Miami,None
Eva,Lucas,Arizona,Coat
Eva,Lucas,Arizona,Shirts
Eva,Lucas,Arizona,Slipper
Farida,Joseph,San Francisco,Coat
Farida,Joseph,San Francisco,Shirts
Farida,Joseph,San Francisco,Shoes
Farida,Joseph,San Francisco,Skirt
Frank,Jacob,Miami,None
Henry,Jackson,Miami,Shoes


## 15. Customer Revenue In March
Calculate the total revenue from each customer in March 2019. 

Output the revenue along with the customer id and sort the results based on the revenue in descending order.

In [34]:
%%sql
SELECT cust_id,
        SUM(case when strftime('%Y-%m',order_date) = '2019-03' then total_order_cost else 0 end) revenue
FROM orders
GROUP BY 1
HAVING revenue > 0
ORDER BY 2 DESC

cust_id,revenue
3,210
15,150
7,80
12,20


In [35]:
%%sql
SELECT cust_id,
        SUM(total_order_cost) revenue
FROM orders
WHERE strftime('%Y-%m',order_date) = '2019-03'
GROUP BY 1
ORDER BY 2 DESC

cust_id,revenue
3,210
15,150
7,80
12,20


In [36]:
%%sql
SELECT cust_id, 
        SUM(total_order_cost) as revenue
FROM orders
WHERE order_date BETWEEN '2019-03-01' AND '2019-03-31'
GROUP BY 1
ORDER BY 2 DESC

cust_id,revenue
3,210
15,150
7,80
12,20


## 16. Host Popularity Rental Prices
You’re given a table of rental property searches by users. The table consists of search results and outputs host information for searchers. Find the minimum, average, maximum rental prices for each host’s popularity rating. The host’s popularity rating is defined as below:
    0 reviews: New
    1 to 5 reviews: Rising
    6 to 15 reviews: Trending Up
    16 to 40 reviews: Popular
    more than 40 reviews: Hot

Tip: The `id` column in the table refers to the search ID. You'll need to create your own host_id by concating price, room_type, host_since, zipcode, and number_of_reviews.

Output host popularity rating and their minimum, average and maximum rental prices.

In [37]:
%%sql
with table1 AS (select price || room_type || DATE(host_since) || zipcode || number_of_reviews AS host_id,
                        price,
                        CASE WHEN number_of_reviews = 0 then 'New'
                        WHEN number_of_reviews BETWEEN 1 AND 5 THEN 'Rising'
                        WHEN number_of_reviews BETWEEN 6 AND 15 THEN 'Trending Up'
                        WHEN number_of_reviews BETWEEN 16 AND 40 THEN 'Popular'
                        WHEN number_of_reviews > 40 THEN 'Hot' END AS host_pop_rating
                from airbnb_search_details)

select host_pop_rating,
        min(price) min_price,
        round(avg(price),2) avg_price,
        max(price) max_price
from table1
group by 1
order by 1

host_pop_rating,min_price,avg_price,max_price
Hot,340.12,464.23,633.51
New,313.55,515.92,741.76
Popular,270.81,472.82,667.83
Rising,355.53,503.85,717.01
Trending Up,361.09,476.28,685.65


## 17. Popularity of Hack
Facebook has developed a new programing language called Hack.To measure the popularity of Hack they ran a survey with their employees. The survey included data on previous programing familiarity as well as the number of years of experience, age, gender and most importantly satisfaction with Hack. Due to an error location data was not collected, but your supervisor demands a report showing average popularity of Hack by office location. Luckily the user IDs of employees completing the surveys were stored.
Based on the above, find the average popularity of the Hack per office location.
Output the location along with the average popularity.

In [38]:
%%sql
select location,
    round(avg(popularity),2) avg_popularity
from facebook_employees t1
left join facebook_hack_survey t2
ON t1.id = t2.employee_id
GROUP BY 1
ORDER BY 2 DESC

location,avg_popularity
India,7.5
USA,4.6
UK,4.33
Switzerland,1.0


## 18. Average Salaries
Compare each employee's salary with the average salary of the corresponding department.
Output the department, first name, and salary of employees along with the average salary of that department.

In [39]:
%%sql
SELECT department,
        first_name,
        salary,
        ROUND(AVG(salary) OVER(PARTITION BY department)) avg_
FROM employee;

department,first_name,salary,avg_
Audit,Shandler,1100,950.0
Audit,Jason,1000,950.0
Audit,Celine,1000,950.0
Audit,Michale,700,950.0
Management,Katty,150000,175000.0
Management,Richerd,250000,175000.0
Management,George,100000,175000.0
Management,Allen,200000,175000.0
Sales,Max,1300,1336.0
Sales,Jennifer,1000,1336.0


## 19. Classify Business Type
Classify each business as either a restaurant, cafe, school, or other. A restaurant should have the word 'restaurant' in the business name. For cafes, either 'cafe', 'café', or 'coffee' can be in the business name. 'School' should be in the business name for schools. All other businesses should be classified as 'other'. Output the business name and the calculated classification.

In [40]:
%%sql
select distinct business_name,
        CASE WHEN LOWER(business_name) LIKE '%school%' THEN 'School'
            WHEN LOWER(business_name) LIKE '%cafe%' OR LOWER(business_name) LIKE '%café%' OR LOWER(business_name) LIKE '%coffee%' THEN 'Cafe'
            WHEN LOWER(business_name) LIKE '%restaurant%' THEN 'Restaurant' 
            ELSE 'Other' END AS business_type
from sf_restaurant_health_violations

business_name,business_type
John Chin Elementary School,School
Sutter Pub and Restaurant,Restaurant
SRI THAI CUISINE,Other
Washington Bakery & Restaurant,Restaurant
Brothers Restaurant,Restaurant
T & L FOOD MARKET,Other
"Antonelli Brothers Meat, Fish, and Poultry Inc.",Other
STARBUCKS COFFEE CO. #603,Cafe
Jiang Ling Cuisine Restaurant,Restaurant
Wing Lee BBQ Restaurant,Restaurant


## 20. Top Cool Votes
Find the review_text that received the highest number of  'cool' votes.
Output the business name along with the review text with the highest numbef of 'cool' votes.

In [41]:
%%sql
with table1 AS (select review_text,
                        business_name,
                        rank() over (order by cool DESC) rnk
                from yelp_reviews)
SELECT business_name,
        review_text
FROM table1
WHERE rnk = 1

business_name,review_text
Roka Akor,"I hate to admit it, but it had been a long while since my last visit to Roka Akor. I deserve a hand slap. But last week, I had the perfect excuse to p"
Lunardis,This is the nicest grocery store in the city. I actually met my wife at this grocery store while shopping for avocados.


In [42]:
%%sql
SELECT business_name,
        review_text
FROM yelp_reviews
WHERE cool = (select max(cool) from yelp_reviews)

business_name,review_text
Roka Akor,"I hate to admit it, but it had been a long while since my last visit to Roka Akor. I deserve a hand slap. But last week, I had the perfect excuse to p"
Lunardis,This is the nicest grocery store in the city. I actually met my wife at this grocery store while shopping for avocados.


## 21. Order Details
Find order details made by Jill and Eva.
Consider the Jill and Eva as first names of customers.
Output the order date, details and cost along with the first name.
Order records based on the customer id in ascending order.

In [43]:
%%sql
SELECT first_name,
        DATE(order_date) order_date,
        order_details,
        total_order_cost
from customers t1
inner join orders t2
on t1.id = t2.cust_id
WHERE first_name IN ('Jill','Eva')

first_name,order_date,order_details,total_order_cost
Jill,2019-02-01,Coat,25
Jill,2019-03-10,Shoes,80
Jill,2019-04-01,Suit,50
Jill,2019-04-02,Skirt,30
Jill,2019-04-03,Dresses,50
Jill,2019-04-04,Coat,25
Jill,2019-04-19,Coat,125
Jill,2019-04-19,Suit,150
Eva,2019-01-11,Coat,125
Eva,2019-01-11,Shirts,60


## 22. Reviews of Categories
Find the top business categories based on the total number of reviews. Output the category along with the total number of reviews. Order by total reviews in descending order.

In [44]:
%%sql
select review_count,
        categories
from yelp_business;

review_count,categories
4,Auto Detailing;Automotive
27,Personal Chefs;Food;Gluten-Free;Food Delivery Services;Event Planning & Services;Restaurants
4,Dry Cleaning & Laundry;Laundry Services;Local Services;Dry Cleaning
3,Wedding Planning;Caterers;Event Planning & Services;Venues & Event Spaces
5,Notaries;Printing Services;Local Services;Shipping Centers
35,Home Services;Television Service Providers;Professional Services;Internet Service Providers;Utilities
3,Bars;Restaurants;Pubs;British;Nightlife
6,Hotels & Travel;Car Rental
13,Fast Food;Sandwiches;Restaurants
10,Rolfing;Health & Medical;Beauty & Spas;Massage


## 23. Highest Salary In Department
Find the employee with the highest salary per department.
Output the department name, employee's first name along with the corresponding salary.

In [45]:
%%sql
select department,
        first_name,
        salary,
        rank() over (partition by department order by salary desc) rnk
from employee

department,first_name,salary,rnk
Audit,Shandler,1100,1
Audit,Jason,1000,2
Audit,Celine,1000,2
Audit,Michale,700,4
Management,Richerd,250000,1
Management,Allen,200000,2
Management,Katty,150000,3
Management,George,100000,4
Sales,Mick,2200,1
Sales,Sarrah,2000,2


In [46]:
%%sql
with table1 as (select department,
                        first_name,
                        salary,
                        rank() over (partition by department order by salary desc) rnk
                from employee)
SELECT department,
        first_name,
        salary
FROM table1
WHERE rnk = 1
ORDER BY 3 DESC

department,first_name,salary
Management,Richerd,250000
Sales,Mick,2200
Audit,Shandler,1100


In [47]:
%%sql
-- not efficient solution for same highest salary of more than one employee ber department 
select department,
        first_name,
        max(salary) salary
from employee
GROUP BY 1
ORDER BY 1,3 DESC

department,first_name,salary
Audit,Shandler,1100
Management,Richerd,250000
Sales,Mick,2200


In [48]:
%%sql
WITH table1 AS (SELECT department, 
                        MAX(salary) salary
                FROM employee         
                GROUP BY department)

SELECT  department,
        first_name,
        salary
FROM employee
WHERE (department,salary) IN table1

department,first_name,salary
Management,Richerd,250000
Sales,Mick,2200
Audit,Shandler,1100


## 24. Lowest Salary In Department
Find the employee with the Lowest salary per department.
Output the department name, employee's first name along with the corresponding salary.

In [49]:
%%sql
with table1 as (select department,
                        first_name,
                        salary,
                        rank() over (partition by department order by salary ASC) rnk
                from employee)
SELECT department,
        first_name,
        salary
FROM table1
WHERE rnk = 1
ORDER BY 1,3 DESC

department,first_name,salary
Audit,Michale,700
Management,George,100000
Sales,Jennifer,1000
Sales,Laila,1000
Sales,Monika,1000
Sales,Joe,1000
Sales,Sam,1000


In [50]:
%%sql
WITH table1 AS (SELECT department, 
                        MIN(salary) salary
                FROM employee         
                GROUP BY department)

SELECT  department,
        first_name,
        salary
FROM employee
WHERE (department,salary) IN table1
ORDER BY 1,3 DESC

department,first_name,salary
Audit,Michale,700
Management,George,100000
Sales,Jennifer,1000
Sales,Laila,1000
Sales,Monika,1000
Sales,Joe,1000
Sales,Sam,1000


## 25. Workers With The Highest Salaries
Find the titles of workers that earn the highest salary. Output the highest-paid title or multiple titles that share the highest salary.

In [51]:
%%sql
with table1 as (select worker_title,
                        salary,
                        rank() over (order by salary DESC) rnk
                from worker
                inner join title
                on worker.worker_id = title.worker_ref_id)
select worker_title AS best_paid_title
from table1
WHERE rnk = 1

best_paid_title
Asst. Manager
Manager


In [52]:
%%sql
with table1 as (select MAX(salary)
                from worker)

select worker_title AS best_paid_title
from worker
inner join title
on worker.worker_id = title.worker_ref_id
WHERE salary IN table1

best_paid_title
Asst. Manager
Manager


## 26. Employee and Manager Salaries
Find employees who are earning more than their managers. Output the employee name along with the corresponding salary.

In [53]:
%%sql
with table1 AS (select id AS manager_id,
                        first_name AS manager_name,
                        salary AS manager_salary
                from employee),
        table2 AS (select id,
                        first_name,
                        manager_id,
                        salary
                from employee)

SELECT first_name,
        salary
FROM table1 t1
INNER JOIN table2 t2
ON t1.manager_id = t2.manager_id
WHERE salary > manager_salary

first_name,salary
Richerd,250000


## 27. Number of violations
You're given a dataset of health inspections. Count the number of violation in an inspection in 'Roxanne Cafe' for each year. If an inspection resulted in a violation, there will be a value in the 'violation_id' column. Output the number of violations by year in ascending order.

In [54]:
%%sql
SELECT strftime('%Y',inspection_date) AS date,
        count(violation_id) AS n_inspections
FROM sf_restaurant_health_violations
WHERE business_name = 'Roxanne Cafe'
GROUP BY 1

date,n_inspections
2015,5
2016,2
2018,3


## 28. Highest Target Under Manager
Find the highest target achieved by the employee or employees who works under the manager id 13. Output the first name of the employee and target achieved. The solution should show the highest target achieved under manager_id=13 and which employee(s) achieved it.

In [55]:
%%sql
WITH table1 AS (SELECT first_name,
                        target,
                        rank() OVER (ORDER BY target DESC) rnk
                FROM salesforce_employees
                WHERE manager_id = 13)
SELECT first_name,
        target
FROM table1
WHERE rnk = 1

first_name,target
Nicky,400
Steve,400
David,400


In [56]:
%%sql
WITH table1 AS (SELECT first_name,
                        target,
                        manager_id,
                        RANK() OVER (PARTITION BY manager_id ORDER BY target DESC) rnk
                FROM salesforce_employees)
SELECT first_name,
        target
FROM table1
WHERE rnk = 1 AND manager_id = 13

first_name,target
Nicky,400
Steve,400
David,400


In [57]:
%%sql
WITH table1 AS (SELECT MAX(target)
                FROM salesforce_employees
                WHERE manager_id = 13)
SELECT first_name,
        target
FROM salesforce_employees
WHERE target IN table1 AND manager_id = 13

first_name,target
Nicky,400
Steve,400
David,400


## 29. Member Count

your query should return the names of the people who are reported to (excluding null values), the number of members that report to them, and the average age of those members as an integer. The rows should be ordered by the names in alphabetical order. Your output should look like the

In [58]:
%%sql
SELECT *
FROM member_count

ID,FirstName,LastName,ReportsTo,Position,Age
1,Daniel,Smith,Bob Boss,Engineer,25
2,Mike,White,Bob Boss,Contractor,22
3,Jenny,Richards,None,CEO,45
4,Robert,Black,Daniel Smith,Sales,22
5,Noah,Fritz,Jenny Richards,Assistant,30
6,David,S,Jenny Richards,Director,32
7,Ashley,Wells,David S,Assistant,25
8,Ashley,Johnson,None,Intern,25


In [59]:
%%sql
SELECT ReportsTo,
        count(ID) AS count_
FROM member_count
GROUP BY 1
HAVING ReportsTo IS NOT NULL

ReportsTo,count_
Bob Boss,2
Daniel Smith,1
David S,1
Jenny Richards,2


In [60]:
%%sql
SELECT ReportsTo,
        count(ID) AS count_
FROM member_count
WHERE ReportsTo IS NOT NULL
GROUP BY 1

ReportsTo,count_
Bob Boss,2
Daniel Smith,1
David S,1
Jenny Richards,2


## 30. User Logins
the table provided shows all new users signing up on a specific date in the format YYYY-MM-DD. Your query should output the change from one month to the next. Because the first month has no preceding month, your output should skip that row. Your output should look like the following table.

In [61]:
%%sql
with table1 as (SELECT  rtrim(substr("January  ,February ,March    ,April    ,May      ,June     ,July     ,August   ,September,October  ,November ,December ", cast(strftime("%m", DateJoined) AS int)*10 - 9, 9)) month_name,
                        strftime("%m", DateJoined) month_,
                        count(ID) current_count,
                        lag(count(ID),1) over (order by strftime("%m", DateJoined) asc) as prev_count
                FROM member_joined
                GROUP BY 1
                ORDER BY 2)

select month_name AS Month,
        current_count - prev_count AS MonthToMonthChange
FROM table1
WHERE prev_count IS NOT NULL

Month,MonthToMonthChange
February,-1
March,6
April,-6
May,4


In [62]:
%%sql
SELECT  case when cast(strftime("%m", DateJoined) AS int) = 1 then 'January'
            when cast(strftime("%m", DateJoined) AS int) = 2 then 'February'
            when cast(strftime("%m", DateJoined) AS int) = 3 then 'March'
            when cast(strftime("%m", DateJoined) AS int) = 4 then 'April'
            when cast(strftime("%m", DateJoined) AS int) = 5 then 'May'
            end AS month,
        count(ID) - lag(count(ID),1) over (order by strftime("%m", DateJoined) asc) AS MonthToMonthChange
FROM member_joined
GROUP BY 1

month,MonthToMonthChange
January,None
February,-1
March,6
April,-6
May,4


## 31. Employee Salaries
your query should return the information for the employee with the third highest salary. Write a query that will find this employee and return that row, but then replace the DivisionID column with the corresponding DivisionName from the table cb_companydivisions. You should also replace the ManagerID column with the ManagerName if the ID exists in the table and is not NULL.                                               

In [63]:
%%sql
SELECT *
FROM cb_salary

ID,Name,DivisionID,ManagerID,Salary
356,Daniel Smith,100,133.0,40000
122,Arnold Sully,101,None,60000
467,Lisa Roberts,100,None,80000
112,Mary Dial,105,467.0,65000
775,Dennis Front,103,None,90000
111,Larry Weis,104,35534.0,75000
222,Mark Red,102,133.0,86000
577,Robert Night,105,12353.0,76000
133,Susan Wall,105,577.0,110000


In [64]:
%%sql
SELECT *
FROM cb_companydivisions

ID,DivisionName
100,Accounting
101,IT
102,Sales
103,Marketing
104,Engineering
105,Customer Support


In [65]:
%%sql
With table1 AS (SELECT ID AS Manager_ID,
                        Name AS Manager_Name
                FROM cb_salary),
        table2 AS (SELECT Name,
                        DivisionName,
                        Manager_Name,
                        Salary,
                        RANK() OVER(ORDER BY Salary DESC) rnk
                FROM cb_salary t1
                INNER JOIN cb_companydivisions t2
                ON t1.DivisionID = t2.ID
                LEFT JOIN table1 t3
                ON t3.Manager_ID = t1.ManagerID)

SELECT Name,
        DivisionName,
        Manager_Name,
        Salary
FROM table2
WHERE rnk = 3


Name,DivisionName,Manager_Name,Salary
Mark Red,Sales,Susan Wall,86000


## 31.1. Employee Count By Divison

In [66]:
%%sql
SELECT t2.DivisionName,
        count(t1.ID) AS EmployeeCount
FROM cb_salary t1
INNER JOIN cb_companydivisions t2
ON t1.DivisionID = t2.ID
GROUP BY 1

DivisionName,EmployeeCount
Accounting,2
Customer Support,3
Engineering,1
IT,1
Marketing,1
Sales,1


## 31.2. Employees With Salary Higher Than Avg. Salary

In [67]:
%%sql
SELECT ID,
        Name
FROM cb_salary
WHERE salary > (SELECT AVG(salary) FROM cb_salary)

ID,Name
467,Lisa Roberts
775,Dennis Front
222,Mark Red
577,Robert Night
133,Susan Wall


## 31.3. Top Three Highest Salaries By Each Division

In [68]:
%%sql
WITH table1 AS (SELECT t1.ID,
                        t1.Name,
                        t2.DivisionName,
                        t1.salary,
                        RANK() OVER (PARTITION BY t2.DivisionName ORDER BY t1.salary DESC) AS rnk
                FROM cb_salary t1
                INNER JOIN cb_companydivisions t2
                ON t1.DivisionID = t2.ID)
SELECT ID,
        Name,
        DivisionName,
        rnk AS Order_,
        salary
FROM table1
WHERE rnk <= 3
ORDER BY 3,4

ID,Name,DivisionName,Order_,salary
467,Lisa Roberts,Accounting,1,80000
356,Daniel Smith,Accounting,2,40000
133,Susan Wall,Customer Support,1,110000
577,Robert Night,Customer Support,2,76000
112,Mary Dial,Customer Support,3,65000
111,Larry Weis,Engineering,1,75000
122,Arnold Sully,IT,1,60000
775,Dennis Front,Marketing,1,90000
222,Mark Red,Sales,1,86000


## 32. Vendor Sort
your query should return the vendor information along with the values from the table cb_vendorinformation. You should combine the values of the two tables based on the GroupID column. The final query should consolidate the rows to be grouped by FirstName, and a Count column should be added at the end that adds up the number of times that person shows up in the table.       
The output table should be sorted by the Count column in ascending order and then sorted by CompanyName in alphabetical order.  

In [69]:
%%sql
SELECT *
FROM cb_vendorinformation

GroupID,CompanyName
35,Shipping & Co.
36,Johnson and Sons
39,News Corp.
40,FireConsulting
5,WaterBus Enterprise
6,Alloy LLC
27,Machinx


In [70]:
%%sql
SELECT *
FROM cb_vendor

GroupID,Year,VendorID,FirstName,LastName,Job,ExternalID,Region
35,2018.0,102.0,Daniel,Knolle,Manager,39765.0,West
35,2018.0,1988.0,Arnold,Sully,Manager,48507.0,West
35,2018.0,1988.0,Arnold,Sully,Manager,48507.0,East
36,2018.0,102.0,Daniel,Knolle,Manager,8219.0,West
36,2018.0,1988.0,Arnold,Sully,Manager,48507.0,West
39,2018.0,102.0,Daniel,Knolle,Manager,8219.0,West
39,2018.0,102.0,Daniel,Knolle,Manager,39765.0,West
39,2018.0,650.0,Lisa,Roberts,Manager,1860.0,West
39,2018.0,650.0,Lisa,Roberts,Manager,5397.0,Central
39,2018.0,1988.0,Arnold,Sully,Manager,48507.0,West


In [71]:
%%sql
SELECT t1.GroupID,
        FirstName,
        LastName,
        Job,
        ExternalID,
        CompanyName,
        Count(*) count
FROM cb_vendor t1
LEFT JOIN cb_vendorinformation t2
ON t1.GroupID = t2.GroupID
GROUP BY FirstName,
        LastName
ORDER BY count ASC,CompanyName

GroupID,FirstName,LastName,Job,ExternalID,CompanyName,count
6,Larry,Weis,Contractor,4219.0,Alloy LLC,2
39,Mary,Dial,Manager,1860.0,News Corp.,2
5,Dennis,S,Contractor,24122.0,WaterBus Enterprise,2
39,Lisa,Roberts,Manager,1860.0,News Corp.,3
35,Arnold,Sully,Manager,48507.0,Shipping & Co.,6
35,Daniel,Knolle,Manager,39765.0,Shipping & Co.,6


## 33. Vendor Join
In this MySQL challenge, your query should return the vendor information along with the values from the table cb_vendorinformation. You should combine the values of the two tables based on the GroupID column. The final query should only print out the GroupID, CompanyName, and final count of all rows that are grouped into each compan name under a column titled Count. The output table should be then sorted by the Count column and then sorted by GroupID so that a higher number appears first.     

In [72]:
%%sql
SELECT t1.GroupID,
        CompanyName,
        Count(*) count
FROM cb_vendor t1
LEFT JOIN cb_vendorinformation t2
ON t1.GroupID = t2.GroupID
GROUP BY 1
ORDER BY count ASC,t1.GroupID DESC

GroupID,CompanyName,count
27,Machinx,1
5,WaterBus Enterprise,1
36,Johnson and Sons,2
35,Shipping & Co.,3
6,Alloy LLC,3
40,FireConsulting,5
39,News Corp.,6
